In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Capstone/text_gcn-master/data/R8')

In [3]:
random_state = None
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
import re, string, unicodedata
import inflect
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords as sw
sww = sw.words()
import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

def load_data(path):
  X, y = [], []
  with open(path, 'r') as myfile:
    for line in myfile:
      label,text = line.split("\t")
      if len(text) == 0:
        continue
      X.append(text)
      y.append(label)
  return X,y

In [0]:
#The dataset has be preprocessed, do not do any preprocessing
X1,y1 = load_data('/content/drive/My Drive/Capstone/text_gcn-master/data/R8/train.txt')
X2,y2 = load_data('/content/drive/My Drive/Capstone/text_gcn-master/data/R8/test.txt')
y1 = np.array(y1)
y2 = np.array(y2)

#Preprocessing

In [0]:
#create contraction dictonary
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

new_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
##expand contractions
def expand_contractions(sentence, contraction_dict = contraction_dict):
  def replace(match):
    return contraction_dict[match.group(0)]
  return new_re.sub(replace, sentence)

In [0]:
def remove_html_tags(textz):
  ##remove html tags
  new_text = []
  new_text = [re.sub(r'<.*?>',' ', x) for x in textz]
  return new_text

def expand_contracts(textz):
  ##expand contractions 
  new_text = []
  for x in textz:
    ##replace all sentences with lowercase
    new = expand_contractions(x.lower())
    new_text.append(new)
  return new_text

In [0]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in sw.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

In [0]:
def preprocessing2(text):
  X1_n = []
  text1 = remove_html_tags(text)
  text2 = expand_contracts(text1)
  for x in text2:
    word_tokens = word_tokenize(x) 
    words = normalize(word_tokens)
    words = lemmatize_verbs(words)
    X1_n.append(words)
  return X1_n

In [0]:
X1_nn = preprocessing2(X1)
X2_nn = preprocessing2(X2)

#Padding

In [0]:
len_list = [len(s) for s in X1_nn]
seq_length = max(len_list)

def add_padding(corpus, seq_length):
    output = []
    for sentence in corpus:
        if len(sentence)>seq_length:
            output.append(sentence[:seq_length])
        else:
            for j in range(seq_length-len(sentence)):
                sentence.append("<PAD>")
            output.append(sentence)
    return output

text_train_pad = add_padding(X1_nn,seq_length)
text_test_pad = add_padding(X2_nn,seq_length)

#Embedding

In [12]:
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-50")

[==================================================] 100.0% 199.5/199.5MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
def get_embeddings(corpus,word_emb_model):
    emb_dim = word_emb_model.vector_size
    out = []
    for sentence in corpus:
        out_temp = []
        for word in sentence:
            try:
                out_temp.append(word_emb_model.wv[word])
            except:
                out_temp.append([0]*emb_dim)
    
        out.append(out_temp)
    return np.array(out)

train_emb = get_embeddings(text_train_pad,word_emb_model)
test_emb = get_embeddings(text_test_pad,word_emb_model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [14]:
train_emb.shape

(5485, 555, 50)

#LSTM

In [15]:
labels = np.unique(y1)

lEnc = LabelEncoder()
lEnc.fit(labels)
label_train_n = lEnc.transform(y1)
label_test_n = lEnc.transform(y2)
numClass = len(labels)

print(labels)
print(lEnc.transform(labels))

['acq' 'crude' 'earn' 'grain' 'interest' 'money-fx' 'ship' 'trade']
[0 1 2 3 4 5 6 7]


In [0]:
n_input = train_emb.shape[2]
n_hidden = 120
n_class = numClass
total_epoch = 2000
learning_rate = 0.01

In [0]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
def make_batch(data,label,size=1000):
    input_batch = []
    target_batch = []
    random_index = np.random.choice(range(len(data)), size, replace=False)
    
    for i in random_index:
        input_batch.append(data[i])
        target_batch.append(label[i])

    return input_batch, target_batch

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(n_input, n_hidden, num_layers=2, batch_first =True)
        self.linear = nn.Linear(n_hidden,n_class)

    def forward(self, x):
        x,_ = self.lstm(x)
        x = self.linear(x[:,-1,:])
        x = F.log_softmax(x, dim=1)
        return x

In [0]:
net = Net().to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

#input_batch_torch = torch.from_numpy(np.array(train_emb)).float().to(device)
#target_batch_torch = torch.from_numpy(np.array(label_train_n)).view(-1).to(device)


for epoch in range(total_epoch):  
    torch.cuda.empty_cache()

    input_dat,output = make_batch(train_emb,label_train_n)
    input_batch_torch = torch.from_numpy(np.array(input_dat)).float().to(device)
    target_batch_torch = torch.from_numpy(np.array(output)).view(-1).to(device)
    net.train()
    outputs = net(input_batch_torch) 
    loss = criterion(outputs, target_batch_torch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    net.eval()
    outputs = net(input_batch_torch) 
    
    if epoch%200 == 199:
        loss = criterion(outputs, target_batch_torch)
        _, predicted = torch.max(outputs, 1)
        acc= accuracy_score(predicted.cpu().numpy(),target_batch_torch.cpu().numpy())

        print('Epoch: %d, loss: %.5f, train_acc: %.2f' %(epoch + 1, loss.item(), acc))

print('Finished Training')

Epoch: 200, loss: 1.14193, train_acc: 0.55
Epoch: 400, loss: 0.59705, train_acc: 0.81
Epoch: 600, loss: 0.35939, train_acc: 0.86
Epoch: 800, loss: 0.32603, train_acc: 0.87
Epoch: 1000, loss: 0.22263, train_acc: 0.92
Epoch: 1200, loss: 0.12932, train_acc: 0.96
Epoch: 1400, loss: 0.04785, train_acc: 0.98
Epoch: 1600, loss: 0.02177, train_acc: 0.99
Epoch: 1800, loss: 0.00590, train_acc: 1.00
Epoch: 2000, loss: 0.00414, train_acc: 1.00
Finished Training


In [0]:
torch.save(net, '/content/drive/My Drive/Anchor/R8_RNN.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
rnn_model = torch.load('/content/drive/My Drive/Anchor/R8_RNN.pt')

In [0]:
test_torch = torch.from_numpy(np.array(test_emb)).float().to(device)
outtt = rnn_model(test_torch)
_, predicted = torch.max(outtt, 1)
y_pre=predicted.cpu().numpy()

In [22]:
from sklearn.metrics import classification_report
print(classification_report(label_test_n, y_pre))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94       696
           1       0.85      0.87      0.86       121
           2       0.97      0.98      0.98      1083
           3       0.57      0.80      0.67        10
           4       0.89      0.72      0.79        81
           5       0.75      0.63      0.69        87
           6       0.51      0.58      0.55        36
           7       0.74      0.87      0.80        75

    accuracy                           0.93      2189
   macro avg       0.78      0.80      0.78      2189
weighted avg       0.93      0.93      0.93      2189



In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [24]:
cnf_matrix = confusion_matrix(label_test_n, y_pre)
cnf_matrix.diagonal()/cnf_matrix.sum(axis=1)

array([0.93965517, 0.8677686 , 0.9833795 , 0.8       , 0.71604938,
       0.63218391, 0.58333333, 0.86666667])

#Anchor

In [25]:
pip install anchor_exp

     |████████████████████████████████| 430kB 2.7MB/s 
     |████████████████████████████████| 276kB 12.6MB/s 
     |████████████████████████████████| 2.0MB 13.2MB/s 
  Created wheel for anchor-exp: filename=anchor_exp-0.0.1.0-cp36-none-any.whl size=434742 sha256=8cc225fc4f8b312e30a0a74bbdffecdc31b70b9d55efe4f9d5b349d7879e3b84
  Stored in directory: /root/.cache/pip/wheels/ff/b7/cd/7bfb36f4a01ff6b1509cd3432f8b208f455d232cee1079e309
  Created wheel for lime: filename=lime-0.2.0.0-cp36-none-any.whl size=284181 sha256=b8370387498fe55561a1e21ca7b2119733eaae3b2773835cd88d89c1b9e5991a
  Stored in directory: /root/.cache/pip/wheels/22/f2/ec/e5ebd07348b2b1ac722e91c2f549fcc220f7d5f25497a61232
Successfully built anchor-exp lime
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


In [26]:
pip install spacy && python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 2.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=01cb678ccbdd8649192fbc3565ff87abe1912e1731645f2b3db3af9dab056ada
  Stored in directory: /tmp/pip-ephem-wheel-cache-avntwa06/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import en_core_web_lg

In [0]:
from anchor import anchor_text
nlp = en_core_web_lg.load()

In [29]:
r8_class = list(labels)
r8_class

['acq', 'crude', 'earn', 'grain', 'interest', 'money-fx', 'ship', 'trade']

In [0]:
explainer = anchor_text.AnchorText(nlp, r8_class, use_unk_distribution=True)

In [0]:
def preprocessing3(text):
  X1_n = []
  text1 = remove_html_tags(text)
  text2 = expand_contracts(text1)
  for x in text2:
    word_tokens = word_tokenize(x) 
    words = remove_non_ascii(word_tokens)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    X1_n.append(words)
  return X1_n

In [0]:
def predict_lr(texts):
  txt = preprocessing3(texts)
  text_pad = add_padding(txt,seq_length)
  text_emb = get_embeddings(text_pad,word_emb_model)
  text_torch = torch.from_numpy(np.array(text_emb)).float().to(device)
  out = rnn_model(text_torch)
  _, predicted = torch.max(out, 1)
  y_pre=predicted.cpu().numpy()

  return y_pre

In [40]:
np.random.seed(1)
text = 'How much is a barrel of oil'
pred = explainer.class_names[predict_lr([text])[0]]
alternative = []
for x in labels:
  if x != pred:
    alternative.append(x)
print('Prediction: %s' % pred)
exp = explainer.explain_instance(text, predict_lr, threshold=0.95)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Prediction: crude


In [41]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Anchor: oil AND a AND of AND much AND How AND barrel
Precision: 1.00

Examples where anchor applies and model predicts crude:

How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil

Examples where anchor applies and model predicts ['acq', 'earn', 'grain', 'interest', 'money-fx', 'ship', 'trade']:

How aware is some aluminum past oil
How impossible is this barrel on oil
How much is an keg for oil
How much is an barrel of oil
How much is the barrel over oil
How much is a barrel through oil
How much is a barrel beyond oil
Where obvious is a barrel across oil
How worried is a barrel on oil
Where much is both holster on oil


#change distribution

In [0]:
explainer = anchor_text.AnchorText(nlp, r8_class, use_unk_distribution=False, use_bert=False)

In [43]:
np.random.seed(1)
text = 'How much is a barrel of oil'
pred = explainer.class_names[predict_lr([text])[0]]
alternative = []
for x in labels:
  if x != pred:
    alternative.append(x)
print('Prediction: %s' % pred)
exp = explainer.explain_instance(text, predict_lr, threshold=0.95)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Prediction: crude


In [44]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Anchor: oil AND a AND of AND much AND How AND barrel
Precision: 1.00

Examples where anchor applies and model predicts crude:

How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil

Examples where anchor applies and model predicts ['acq', 'earn', 'grain', 'interest', 'money-fx', 'ship', 'trade']:

How aware is some aluminum past oil
How impossible is this barrel on oil
How much is an keg for oil
How much is an barrel of oil
How much is the barrel over oil
How much is a barrel through oil
How much is a barrel beyond oil
Where obvious is a barrel across oil
How worried is a barrel on oil
Where much is both holster on oil


In [45]:
print('Partial anchor: %s' % (' AND '.join(exp.names(0))))
print('Precision: %.2f' % exp.precision(0))
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Partial anchor: oil
Precision: 0.66

Examples where anchor applies and model predicts crude:

HOW much is a nose of oil
Where much is a barrel of oil
How interesting is a barrel of oil
Why much is an barrel of oil
WHERE glad is any bolt beyond oil
How impossible is this tubing in oil
How great is a firearm throughout oil
How much is a barrel towards oil
Where much is every barrel of oil
HOW glad is a pump with oil

Examples where anchor applies and model predicts ['acq', 'earn', 'grain', 'interest', 'money-fx', 'ship', 'trade']:

How aware is some aluminum past oil
How impossible is this barrel on oil
How much is an keg for oil
How much is an barrel of oil
How much is the barrel over oil
How much is a barrel through oil
How much is a barrel beyond oil
Where obvious is a barrel across oil
How worried is a barrel on oil
Where much is both holster on oil


#Using BERT

In [0]:
pip install torch transformers spacy && python -m spacy download en_core_web_sm

     |████████████████████████████████| 573kB 4.8MB/s 
     |████████████████████████████████| 1.0MB 54.9MB/s 
     |████████████████████████████████| 890kB 42.1MB/s 
     |████████████████████████████████| 3.7MB 39.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=01496d036419212c4833fa91582cd9b1a0afeb22ab9f55e2d8923353ce5fa793
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [0]:
explainer = anchor_text.AnchorText(nlp, r8_class, use_unk_distribution=False, use_bert=True)

In [0]:
np.random.seed(1)
text = 'How much is a barrel of oil'
pred = explainer.class_names[predict_lr([text])[0]]
alternative = []
for x in labels:
  if x != pred:
    alternative.append(x)
print('Prediction: %s' % pred)

exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Prediction: crude


In [0]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Anchor: oil AND How AND is AND a AND much AND barrel AND of
Precision: 1.00

Examples where anchor applies and model predicts crude:

How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil

Examples where anchor applies and model predicts ['acq', 'earn', 'grain', 'interest', 'money-fx', 'ship', 'trade']:




In [0]:
print('Partial anchor: %s' % (' AND '.join(exp.names(0))))
print('Precision: %.2f' % exp.precision(0))
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_same_prediction=True)]))
print()

print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=0, only_different_prediction=True)]))

Partial anchor: oil
Precision: 0.49

Examples where anchor applies and model predicts crude:

• • divers ##ified tanker of oil
after it ##eration any synthetic edible oil
bell ##ows ^ a jar of oil
・ र ##भ ##ট ##ر + oil
• increasing marginal oil potential of oil
How ##ards becomes a star of oil
• petrol - a barrel olive oil
lit ##hium ##ulus / # 7 oil
crude oil is a type of oil
am ##bula ##bang ##ah oil of oil

Examples where anchor applies and model predicts ['acq', 'earn', 'grain', 'interest', 'money-fx', 'ship', 'trade']:

salt sour ##e a ##qua ##ck oil
( di ##sso ##ch ##romatic synthetic oil
• • identifying a package of oil
petrol ##ogy ##ji ##ggle color of oil
p ##oly ##mp ##g ##r ##af oil
fat - soluble serum fatty acid oil
c ##est ##imator ##ific side of oil
How ##don ##ned a tin pot oil
• de ##ce = lifetime of oil
2006 – 07 % extraction of oil


In [0]:
np.random.seed(1)
text = 'How much is a barrel of oil'
pred = explainer.class_names[predict_lr([text])[0]]
alternative = []
for x in labels:
  if x != pred:
    alternative.append(x)
print('Prediction: %s' % pred)

exp = explainer.explain_instance(text, predict_lr, threshold=0.95, verbose=False, onepass=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


Prediction: crude


In [0]:
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(only_different_prediction=True)]))

Anchor: oil AND How AND is AND a AND much AND barrel AND of
Precision: 1.00

Examples where anchor applies and model predicts crude:

How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil
How much is a barrel of oil

Examples where anchor applies and model predicts ['acq', 'earn', 'grain', 'interest', 'money-fx', 'ship', 'trade']:




In [0]:
print('Partial anchor: %s' % (' AND '.join(exp.names(0))))
print('Precision: %.2f' % exp.precision(0))
print()
print('Examples where anchor applies and model predicts %s:' % pred)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=2, only_same_prediction=True)]))
print()
print('Examples where anchor applies and model predicts %s:' % alternative)
print()
print('\n'.join([x[0] for x in exp.examples(partial_index=2, only_different_prediction=True)]))

Partial anchor: oil
Precision: 0.47

Examples where anchor applies and model predicts crude:

How ##dock is a port of oil
How sulfur is a crude conventional oil
How oil is developing reservoirs of oil
How ##lo is a barrel of oil
How milk is a minimal palm oil
How ##l is soft barrel petroleum oil
How that is ##rae for crude oil
How use is replacing barrel in oil
How oil is flat barrel ##ized oil
How ##a is ##rae ##fa lime oil

Examples where anchor applies and model predicts ['acq', 'earn', 'grain', 'interest', 'money-fx', 'ship', 'trade']:

How ##itzer is a barrel of oil
How ##u is ##ol ##rab cane oil
How petrol is ##hp fuel olive oil
How ##tock is a source of oil
How ##man is banned community of oil
How ##wood is ##mail by salmon oil
How ##men is a grain of oil
How ##ron is chemical reservoir of oil
How petroleum is a palm olive oil
How ##ion is a kind of oil


In [46]:
exp.show_in_notebook()

Output hidden; open in https://colab.research.google.com to view.

In [39]:
exp.show_in_notebook()

IndexError: ignored